In [1]:
import numpy as np
import os
import tensorflow as tf
from keras.datasets import mnist

import autokeras as ak
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from IPython.display import Image as IPImage, display  # for displaying images in Jupyter


# File paths for the Fake and Real image directories
fake_dir = '/home/michael/Documenti/Milinda_Githubproject/Weibo and Twitter/twitter/images_train/fake_images/'
real_dir = '/home/michael/Documenti/Milinda_Githubproject/Weibo and Twitter/twitter/images_train/real_images/'

# Prepare the data
def prepare_data(fake_dir, real_dir, img_size=(128, 128)):
    images = []
    image_paths = []
    labels = []

    # Fake images
    for img_name in os.listdir(fake_dir):
        if img_name.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            img_path = os.path.join(fake_dir, img_name)
            img = image.load_img(img_path, target_size=img_size)
            img = image.img_to_array(img)
            img /= 255.0  # Normalize the images
            images.append(img)
            labels.append(0)  # Fake = 0
            image_paths.append(img_path)

    # Real images
    for img_name in os.listdir(real_dir):
        if img_name.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            img_path = os.path.join(real_dir, img_name)
            img = image.load_img(img_path, target_size=img_size)
            img = image.img_to_array(img)
            img /= 255.0  # Normalize the images
            images.append(img)
            labels.append(1)  # Real = 1
            image_paths.append(img_path)

    # Convert to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels, image_paths

# Load and prepare the data
x_data, y_data, image_paths = prepare_data(fake_dir, real_dir)

# Split the data into training and testing sets (80/20 split)
x_train, x_test, y_train, y_test, image_paths_train, image_paths_test = train_test_split(x_data, y_data, image_paths, test_size=0.2, random_state=42)

# Initialize the AutoKeras ImageClassifier
clf = ak.ImageClassifier(overwrite=True, max_trials=3)  # max_trials sets the number of models to try

# Train the classifier
clf.fit(x_train, y_train, epochs=5) # Increase epochs to train longer if needed




Trial 3 Complete [00h 03m 04s]
val_loss: 0.6924039125442505

Best val_loss So Far: 0.6473692655563354
Total elapsed time: 00h 03m 54s
Epoch 1/5
10/10 [==============================] - 1s 78ms/step - loss: 2.0009 - accuracy: 0.5172
Epoch 2/5
10/10 [==============================] - 1s 77ms/step - loss: 0.6793 - accuracy: 0.5586
Epoch 3/5
10/10 [==============================] - 1s 76ms/step - loss: 0.6127 - accuracy: 0.6621
Epoch 4/5
10/10 [==============================] - 1s 76ms/step - loss: 0.5396 - accuracy: 0.7828
Epoch 5/5
10/10 [==============================] - 1s 81ms/step - loss: 0.4169 - accuracy: 0.8276
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


In [7]:
# Evaluate the classifier
test_loss, test_acc = clf.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")
# Evaluate the model on the test data
test_loss, test_acc = clf.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.3f}")

# Predict with the best model
y_pred = clf.predict(x_test)
print(f"Predicted labels: {y_pred[:5].flatten()}")
print(f"True labels: {y_test[:5]}")

3/3 [==============================] - 0s 14ms/step - loss: 0.5954 - accuracy: 0.6849
Test accuracy: 0.6849315166473389


3/3 [==============================] - 0s 13ms/step - loss: 0.5954 - accuracy: 0.6849
Test Accuracy: 0.685
3/3 [==============================] - 0s 18ms/step
Predicted labels: [0. 1. 0. 0. 0.]
True labels: [0 0 0 1 0]


In [8]:
keras_model=clf.export_model()
keras_model.summary()


# Evaluate save file
#predictor.save(

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 cast_to_float32 (CastToFlo  (None, 128, 128, 3)       0         
 at32)                                                           
                                                                 
 normalization (Normalizati  (None, 128, 128, 3)       7         
 on)                                                             
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 124, 124, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 62, 62, 64)        0     

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.6849
Precision: 0.7143
Recall: 0.4688
F1 Score: 0.5660


In [6]:
# Take one image from the test data, show it, and display the label and prediction
sample_index = 984 # You can change this index to display a different image

sample_image_path = image_paths_test[sample_index]
true_label = y_test[sample_index]

# Convert predicted_label to a scalar if it's a numpy array
predicted_label = y_pred[sample_index]
if isinstance(predicted_label, np.ndarray):
    predicted_label = predicted_label.item()

# Convert label numbers to human-readable labels
label_map = {0: 'Fake', 1: 'Real'}
true_label_text = label_map[true_label]
predicted_label_text = label_map[predicted_label]

# Display the image
display(IPImage(filename=sample_image_path))

# Print the true label and predicted label
print(f"True Label: {true_label_text}")
print(f"Predicted Label: {predicted_label_text}")


IndexError: list index out of range

In [2]:
import numpy as np
import tensorflow as tf
from keras.datasets import mnist

import autokeras as ak




2024-09-04 17:41:45.118597: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-04 17:41:45.139475: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 17:41:45.445546: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
!pip install autokeras

In [8]:
!pip install pillow

  Using cached pillow-10.4.0-cp39-cp39-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp39-cp39-manylinux_2_28_x86_64.whl (4.5 MB)


In [2]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert

try:
    print("TensorRT is available in TensorFlow.")
    print(trt_convert)
except ImportError:
    print("TensorRT is not available in TensorFlow.")



TensorRT is available in TensorFlow.
<module 'tensorflow.python.compiler.tensorrt.trt_convert' from '/home/michael/miniconda3/envs/autoKeras/lib/python3.9/site-packages/tensorflow/python/compiler/tensorrt/trt_convert.py'>


In [11]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
import autokeras as ak
from tensorflow.keras.utils import to_categorical

# File paths for the Fake and Real image directories
fake_dir = '/home/michael/Documenti/Milinda_Githubproject/Weibo and Twitter/twitter/images_train/fake_images/'
real_dir = '/home/michael/Documenti/Milinda_Githubproject/Weibo and Twitter/twitter/images_train/real_images/'
file_path = '/home/michael/Documenti/Milinda_Githubproject/Weibo and Twitter/twitter/train_posts_twitter.csv'

# Prepare image data
def prepare_data(fake_dir, real_dir, img_size=(128, 128)):
    images = []
    labels = []

    # Fake images
    for img_name in os.listdir(fake_dir):
        if img_name.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            img_path = os.path.join(fake_dir, img_name)
            img = image.load_img(img_path, target_size=img_size)
            img = image.img_to_array(img) / 255.0  # Normalize images
            images.append(img)
            labels.append(0)  # Fake = 0

    # Real images
    for img_name in os.listdir(real_dir):
        if img_name.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            img_path = os.path.join(real_dir, img_name)
            img = image.load_img(img_path, target_size=img_size)
            img = image.img_to_array(img) / 255.0  # Normalize images
            images.append(img)
            labels.append(1)  # Real = 1

    # Convert to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels

# Load the image data
x_image_data, y_image_labels = prepare_data(fake_dir, real_dir)

# Prepare the text data
df = pd.read_csv(file_path)
df_expanded = df.assign(image_id=df['image_id(s)'].astype(str).str.split(','))
df_expanded = df_expanded.explode('image_id').reset_index(drop=True)

# Create text and image ID mapping
text_data = df_expanded['post_text']
image_ids = df_expanded['image_id']
text_labels = df_expanded['label']  # Assuming the label column exists and contains 0 for "fake" and 1 for "real"

# Convert text labels to categorical
text_labels = np.array(text_labels)

# Split both text and image data into train/test sets
x_image_train, x_image_test, y_image_train, y_image_test = train_test_split(
    x_image_data, y_image_labels, test_size=0.2, random_state=42)

x_text_train, x_text_test, y_text_train, y_text_test = train_test_split(
    text_data, text_labels, test_size=0.2, random_state=42)

# Create the AutoKeras multimodal model
input_image = ak.ImageInput()
input_text = ak.TextInput()

# Combine the two inputs
merged_output = ak.Merge()([input_image, input_text])
output = ak.ClassificationHead()(merged_output)

# Build the AutoKeras model
model = ak.AutoModel(
    inputs=[input_image, input_text],
    outputs=output,
    overwrite=True,
    max_trials=5
)

# Fit the model
model.fit(
    [x_image_train, x_text_train],
    y_image_train,  # You can use the same labels for both modalities
    epochs=10
)

# Evaluate the model
loss, accuracy = model.evaluate([x_image_test, x_text_test], y_image_test)
print(f"Test Accuracy: {accuracy:.2f}")


TypeError: Expect the data to TextInput to be numpy.ndarray or tf.data.Dataset, but got <class 'pandas.core.series.Series'>.

In [22]:

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
import autokeras as ak
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# File paths
fake_dir = '/home/michael/Documenti/Milinda_Githubproject/Weibo and Twitter/twitter/images_train/fake_images/'
real_dir = '/home/michael/Documenti/Milinda_Githubproject/Weibo and Twitter/twitter/images_train/real_images/'
file_path = '/home/michael/Documenti/Milinda_Githubproject/Weibo and Twitter/twitter/Rearranged_Train_Posts_Single_ImageID.csv'

# Helper function to load and preprocess image data
def load_images(image_dir, img_size=(128, 128)):
    image_dict = {}
    for img_name in os.listdir(image_dir):
        if img_name.endswith(('.png', '.jpg', '.jpeg', '.gif')):
            # Remove the file extension to get image_id
            image_id = os.path.splitext(img_name)[0]
            img_path = os.path.join(image_dir, img_name)
            img = image.load_img(img_path, target_size=img_size)
            img_array = image.img_to_array(img) / 255.0  # Normalize images
            image_dict[image_id] = img_array
    return image_dict

# Load image data
fake_images = load_images(fake_dir)
real_images = load_images(real_dir)

# Combine fake and real images
all_images = {**fake_images, **real_images}

# Load and preprocess the text data
df = pd.read_csv(file_path)

# Remove rows with null `image_id` or those that don't have a matching image
df = df.dropna(subset=['image_id'])
df['image_id'] = df['image_id'].astype(str)  # Ensure image_id is string for matching
df = df[df['image_id'].isin(all_images.keys())]

# Extract the matched images and labels
image_data = np.array([all_images[img_id] for img_id in df['image_id']])
text_data = df['post_text'].tolist()  # Leave as a list for AutoKeras tokenization
labels = df['label'].values

# Split the data into training and testing sets
x_image_train, x_image_test, x_text_train, x_text_test, y_train, y_test = train_test_split(
    image_data, text_data, labels, test_size=0.2, random_state=42
)

# Ensure text data is passed as a NumPy array of strings
x_text_train = np.array(x_text_train, dtype=str)
x_text_test = np.array(x_text_test, dtype=str)

# Create the AutoKeras multimodal model
input_image = ak.ImageInput()
input_text = ak.TextInput()

# Add preprocessing for the text input
preprocessed_text = ak.TextToNgramVector()(input_text)  # Tokenize and vectorize text
merged_output = ak.Merge()([input_image, preprocessed_text])
output = ak.ClassificationHead()(merged_output)

# Build the AutoKeras model
model = ak.AutoModel(
    inputs=[input_image, input_text],
    outputs=output,
    overwrite=True,
    max_trials=5
)

# Fit the model
model.fit(
    [x_image_train, x_text_train],
    y_train,
    epochs=10
)

# Evaluate the classifier
test_loss, test_acc = model.evaluate([x_image_test, x_text_test], y_test)
print(f"Test Accuracy: {test_acc:.4f}")




Trial 5 Complete [00h 00m 10s]
val_loss: 0.011727719567716122

Best val_loss So Far: 0.011041399091482162
Total elapsed time: 00h 00m 53s
Epoch 1/10
299/299 [==============================] - 1s 2ms/step - loss: 0.2968 - accuracy: 0.9338
Epoch 2/10
299/299 [==============================] - 1s 2ms/step - loss: 0.0229 - accuracy: 0.9952
Epoch 3/10
299/299 [==============================] - 1s 2ms/step - loss: 0.0222 - accuracy: 0.9934
Epoch 4/10
299/299 [==============================] - 1s 2ms/step - loss: 0.0163 - accuracy: 0.9949
Epoch 5/10
299/299 [==============================] - 1s 2ms/step - loss: 0.0174 - accuracy: 0.9947
Epoch 6/10
299/299 [==============================] - 1s 2ms/step - loss: 0.0082 - accuracy: 0.9974
Epoch 7/10
299/299 [==============================] - 1s 2ms/step - loss: 0.0085 - accuracy: 0.9974
Epoch 8/10
299/299 [==============================] - 1s 2ms/step - loss: 0.0170 - accuracy: 0.9950
Epoch 9/10
299/299 [==============================] - 1s 2ms/s

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


75/75 [==============================] - 0s 1ms/step - loss: 0.0308 - accuracy: 0.9954
Test Accuracy: 0.9954


In [23]:


# Predict with the best model
y_pred = model.predict([x_image_test, x_text_test])
y_pred_labels = np.argmax(y_pred, axis=1)  # Convert predictions to class labels

print(f"Predicted labels: {y_pred_labels[:5].flatten()}")
print(f"True labels: {y_test[:5]}")

75/75 [==============================] - 0s 2ms/step
Predicted labels: [0 0 0 0 0]
True labels: ['real' 'real' 'fake' 'real' 'real']


In [25]:
# Export the best model and print its summary
keras_model = model.export_model()
keras_model.summary()
# Define the mapping between numeric labels and string labels
label_mapping = {0: 'fake', 1: 'real'}

# Map numeric predictions back to string labels
y_pred_mapped = np.array([label_mapping[pred] for pred in y_pred_labels])

# Ensure y_test is a NumPy array for consistency
y_test = np.array(y_test)

# Evaluate using sklearn metrics
accuracy = accuracy_score(y_test, y_pred_mapped)
precision = precision_score(y_test, y_pred_mapped, pos_label='real')
recall = recall_score(y_test, y_pred_mapped, pos_label='real')
f1 = f1_score(y_test, y_pred_mapped, pos_label='real')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


# Optionally save the model
keras_model.save("best_autokeras_model.h5")
print("Best model saved as 'best_autokeras_model.h5'")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None,)]                    0         []                            
                                                                                                  
 cast_to_float32 (CastToFlo  (None, 128, 128, 3)          0         ['input_1[0][0]']             
 at32)                                                                                            
                                                                                                  
 expand_last_dim (ExpandLas  (None, 1)                    0         ['input_2[0][0]']         

/home/michael/miniconda3/envs/autoML/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/michael/miniconda3/envs/autoML/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NotImplementedError: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Received a model or layer TextVectorization with weights [<tf.Variable 'Variable:0' shape=(None,) dtype=float32, numpy=
array([7.7119904, 0.884729 , 1.73271  , ..., 8.473241 , 8.473241 ,
       8.473241 ], dtype=float32)>, <keras.src.layers.preprocessing.index_lookup.VocabWeightHandler object at 0x74866eb8cd60>, <tf.Variable 'Variable:0' shape=() dtype=int64, numpy=0>]